<a href="https://colab.research.google.com/github/wlrma0108/HuggingFace/blob/main/Adapter%20version_Error.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -U adapter-transformers
!pip install datasets

     |████████████████████████████████| 3.2 MB 5.3 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 895 kB 42.2 MB/s 
     |████████████████████████████████| 3.3 MB 36.5 MB/s 
     |████████████████████████████████| 596 kB 46.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 312 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 35.7 MB/s 
     |████████████████████████████████| 212 kB 46.8 MB/s 
     |████████████████████████████████| 134 kB 47.9 MB/s 
     |████████████████████████████████| 127 kB 45.0 MB/s 
     |████████████████████████████████| 271 kB 49.1 MB/s 
     |████████████████████████████████| 144 kB 50.5 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [5]:
from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")
dataset.num_rows

Downloading:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/921 [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/488k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset rotten_tomatoes_movie_review downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'test': 1066, 'train': 8530, 'validation': 1066}

In [6]:
dataset['train'][0]

{'label': 1,
 'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .'}

In [7]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
def encode_batch(batch):
  return tokenizer(batch["text"],max_length=80,truncation=True, padding="max_length")

dataset=dataset.map(encode_batch,batched=True)
dataset.rename_column_("label","labels")
dataset.set_format(type="torch",columns=["input_ids","attention_mask","labels"])

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: rename_column_ is deprecated and will be removed in the next major version of datasets. Use DatasetDict.rename_column instead.
  import sys


In [8]:
from transformers import RobertaConfig,RobertaModelWithHeads

config=RobertaConfig.from_pretrained("roberta-base",num_labels=2,)
model= RobertaModelWithHeads.from_pretrained("roberta-base",config=config,)



Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

In [15]:

#model.add_classification_head("rotten_tomatoes",num_labels=2,id2label={0:"bad",1:"good"})
#model.train_adapter("rotten_tamatoes")

In [21]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args=TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    remove_unused_columns=False,

)
def compute_accuracy(p: EvalPrediction):
  preds=np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}
trainer=AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset = dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


ValueError: ignored

In [18]:
trainer.train()

NameError: ignored

In [19]:
trainer.evaluate()


NameError: ignored